## How to Interact with APIs Using Function Calling in Gemini


### 1. Overview

**What is function calling in Gemini?**

The [Vertex AI Gemini API](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/overview) is a family of generative AI models developed by Google DeepMind that is designed for multimodal use cases. [Function calling](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling) is a feature of Gemini models that makes it easier for developers to get structured data outputs from generative models.

Developers can then use these outputs to call other APIs and return the relevant response data to the model. In other words, function calling helps you connect your generative models to external systems so that the generated content includes the most up-to-date and accurate information.

**How function calling works**

Functions are described using function declarations, which helps the generative model understand the purpose and parameters within a function. After you pass function declarations in a query to a generative model, the model returns a structured object that includes the names of relevant functions and their arguments based on the user's query. Note that with function calling, the model doesn't actually call the function. Instead, you can use the returned function and parameters to call the function in any language, library, or framework that you'd like!

**What you'll build**

In this codelab, you'll build a generative AI pipeline with the Vertex AI Gemini API and Python. Using your app, users can ask about exchange rates, and the system will fetch the latest data from an external API and respond to the user with the answer.

**What you'll learn**

- How to interact with the Gemini model using the Python client library
- How to define a function declaration and register it as a tool
- How to call Gemini and get a function call response
- How to return the function response to Gemini and respond to the user


### 2. Setup and requirements

- Enable Vertex AI API
- Install Python client library for Vertex AI


In [1]:
! pip install --upgrade --quiet google-cloud-aiplatform

### 3. Understand the problem

Have you ever interacted with a large language model or generative AI model and asked it about real-time or current information, only to get a response with outdated information or inaccurate information?

Let's try it now! First, we'll import the relevant Python packages and initialize the Gemini model. You can run the following code in a Python development environment such as Colab or Colab Enterprise and by installing the latest version of the Vertex AI SDK for Python:


In [5]:
import os
import vertexai
from vertexai.generative_models import GenerativeModel

# from google.auth import default

# if os.getenv("TRAINING_CREDENTIALS") is not None:
#     print("authenticate with a service account...")
#     os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.environ["TRAINING_CREDENTIALS"]

# credentials, project = default(quota_project_id=os.environ["TRAINING_PROJECT_ID"])

# define project and location details
PROJECT_ID = os.environ["TRAINING_PROJECT_ID"]
LOCATION = "us-central1"

vertexai.init(project=PROJECT_ID, location=LOCATION)

model = GenerativeModel("gemini-1.5-pro-001")

Now, let's ask a question about the exchange rate for different currencies today:


In [6]:
response = model.generate_content(
    "What's the exchange rate for euros to dollars today?"
)
print(response.text)

I do not have access to real-time information, including constantly fluctuating data like currency exchange rates. 

To get the most up-to-date exchange rate for euros to dollars, I recommend checking a reliable online source such as:

* **Google Finance:** Search "EUR to USD" on Google
* **XE.com:** This website specializes in currency conversions
* **Your Bank's Website:** Many banks provide current exchange rates for their customers

Please keep in mind that exchange rates are constantly changing, and the rate you see may differ slightly depending on the provider and any associated fees. 



If an end-user received this type of response, they would need to switch contexts to look up the currencies that they're interested in, fetch the latest exchange rate, and perform any conversions on their own.

Ideally, a generative model pipeline could handle some or all of these tasks for the user. In the next section, you'll try some common workarounds for getting structured responses from generative models so that you can call external systems.


### 4. Try common workarounds

When working with generative models in scenarios where you need up-to-date information or data from external sources, you could call an external API then feed the results back to the generative model for it to use in its response.

Before you call an external system, you need to determine the right function to use, extract the relevant parameters from the user, and put the parameters into a structured data object. This usually involves exhaustive prompt engineering to coerce the generative model to output valid structured data.

Let's revisit the question that we asked in the previous section and add some additional instructions for the model. Try sending the following request to the Gemini model:


In [7]:
user_prompt = "What's the exchange rate from euros to US dollars today?"

response = model.generate_content(
    """
Your task is to extract parameters from the user's input and return it as a
structured JSON payload. The user will ask about the exchange rate and which
currency they are converting from and converting to.

User input: {user_prompt}

Please extract the currencies as parameters and put them in a JSON object.
""".format(user_prompt=user_prompt)
)
print(response.text)

```json
{
  "from": "euros",
  "to": "US dollars"
}
``` 



In particular, the first and last lines of the text response include backticks to delimit the code block, the first line includes a language specifier, and the values in the JSON object are not the standard three-letter currency abbreviations that a currency exchange API would expect as input parameters.

We could try to use Python to post-process this text into valid JSON and a dictionary, add more instructions to the prompt, provide one or more examples of desired output, fine-tune the model, or make another call to the generative model asking it to clean up the JSON.

But there is a more deterministic way! Let's learn how to use function calling in Gemini to query for information in external services and return relevant responses to end-users.


### 5. How function calling works

Before we get started with parameter extraction and function calling, let's walk through the steps of function calling and which components are used at runtime.

![](./images/gemini-function-calling-overview_1920.png)

**User input to Gemini API**

The prompt from the user is sent to the Gemini API, and in that API call to the Gemini model, the developer has defined one or more function declarations within a tool so that the Gemini model knows which functions it can call and how to call them.

> User Input -> Gemini API -> Function Call

**The Gemini API returns a Function Call**

Based on the content of the user input and prompt, Gemini will return a Function Call response with structured data that includes the name of the function to call and the corresponding parameters to use.

> Gemini API -> Function Call

**Make an API request**

Then, you'll use the function name and parameters to make an API request to retrieve information from an external system or API. This API request and response is implemented by the developer in the application code and happens outside of the scope of the Gemini API and SDK. For example, you might use the requests library in Python to call a REST API and receive a JSON response. Or you can call the function using your preferred approach and client library.

> API Parameters -> External API -> API Response

**Return the API Response to Gemini**

Finally, you'll pass the API response back to the Gemini model so that it can generate a response to the end-user's initial prompt or invoke another Function Call response if the Gemini model determines that it needs additional information.

> API Response -> Function Response -> Model Output


### 6. Choose your API

Now that you understand the overall flow and specific steps in function calling, you'll build a generative AI pipeline to fetch the latest currency exchange rates. First, we'll need to select which API that we want to use as a source of information.

For our currency exchange app, we'll use the REST API at https://www.frankfurter.app/ to fetch the latest information about global exchange rates.

To interact with this REST API, we might make a REST API call with `requests` in Python as:


In [8]:
import requests

url = "https://api.frankfurter.app/latest"
response = requests.get(url)
response.text

'{"amount":1.0,"base":"EUR","date":"2025-01-22","rates":{"AUD":1.6603,"BGN":1.9558,"BRL":6.2738,"CAD":1.4972,"CHF":0.9449,"CNY":7.5892,"CZK":25.139,"DKK":7.4612,"GBP":0.84466,"HKD":8.1322,"HUF":411.08,"IDR":16929,"ILS":3.6962,"INR":90.21,"ISK":146.5,"JPY":162.72,"KRW":1496.01,"MXN":21.471,"MYR":4.633,"NOK":11.7525,"NZD":1.8387,"PHP":61.003,"PLN":4.23,"RON":4.9768,"SEK":11.467,"SGD":1.4126,"THB":35.334,"TRY":37.231,"USD":1.0443,"ZAR":19.2877}}'

or a `cURL` request such as:

```bash
curl https://api.frankfurter.app/latest
```


Because function calling in Gemini does not actually make the external API call for you, there are no such restrictions on what type of API that you use! You could use a Cloud Run Service, a Cloud Function, an API request to a Google Cloud service, or any external REST API.


### 7. Define a function and tool

Now that you've selected a REST API to use, we can now define an API specification and register the function in a tool.

Ensure that you've installed the latest version of the [Vertex AI SDK for Python](https://cloud.google.com/vertex-ai/docs/python-sdk/use-vertex-ai-python-sdk).

Then, import the necessary modules from the Python SDK and initialize the Gemini model:


In [9]:
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

model = GenerativeModel("gemini-1.5-pro-001")

Referring back to the REST API at https://api.frankfurter.app/, we can see that it accepts the following input parameters:

| Parameter | Type   | Description                         |
| --------- | ------ | ----------------------------------- |
| from      | String | Currency to convert from            |
| to        | String | Currency to convert to              |
| date      | String | Date to fetch the exchange rate for |

Using these parameters, a partial OpenAPI specification for this REST API in YAML format looks like:

```yaml
openapi: 3.0.0
info:
  title: Frankfurter Exchange Rate API
  description: This API provides current and historical exchange rates
  version: 1.0.0
servers:
  - url: https://api.frankfurter.app
paths:
  /{date}:
    get:
      summary: Get the latest currency exchange rates.
      parameters:
        - name: date
          in: path
          description: Get currency rates for a specific date or 'latest' if a date is not specified
          required: true
          schema:
            type: string
        - name: from
          in: query
          description: The currency to convert from.
          required: true
          schema:
            type: string
        - name: to
          in: query
          description: The currency to convert to.
          schema:
            type: string
```


Now, let's register this as a `FunctionDeclaration` using the Python SDK for Gemini:


In [12]:
get_exchange_rate_func = FunctionDeclaration(
    name="get_exchange_rate",
    description="Get the exchange rate for currencies between countries",
    parameters={
        "type": "object",
        "properties": {
            "currency_date": {
                "type": "string",
                "description": "A date that must always be in YYYY-MM-DD format or the value 'latest' if a time period is not specified",
            },
            "currency_from": {
                "type": "string",
                "description": "The currency to convert from in ISO 4217 format",
            },
            "currency_to": {
                "type": "string",
                "description": "The currency to convert to in ISO 4217 format",
            },
        },
        "required": [
            "currency_from",
            "currency_date",
        ],
    },
)

Be sure to use as much detail as possible in the function and parameter descriptions since the generative model will use this information to determine which function to select and how to fill the parameters in the function call.

Finally, you'll define a`Tool` that includes the function declaration:


In [13]:
exchange_rate_tool = Tool(
    function_declarations=[get_exchange_rate_func],
)

Here, you're using one function declaration within a tool, but note that you can register one or more function declarations in a tool, and the model will select the appropriate function to use at runtime. Refer to the documentation on [Function Calling in the Gemini API](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling) for more details on the `FunctionDeclaration`, `Tool`, and related classes in the Gemini SDK for Python.

You've completed the configuration of your function and tool definitions. In the next section, we'll call the generative model with this tool and get back a function call that we can use to call the REST API.


### 8. Generate a function call

Now you can prompt the generative model and include the `tool` that you defined:


In [18]:
prompt = """What is the exchange rate from Australian dollars to Swedish krona?
How much is 500 Australian dollars worth in Swedish krona?"""

response = model.generate_content(
    prompt,
    tools=[exchange_rate_tool],
)

Let's take a look at the response object:


In [19]:
print(response.candidates[0].content)

role: "model"
parts {
  function_call {
    name: "get_exchange_rate"
    args {
      fields {
        key: "currency_to"
        value {
          string_value: "SEK"
        }
      }
      fields {
        key: "currency_from"
        value {
          string_value: "AUD"
        }
      }
      fields {
        key: "currency_date"
        value {
          string_value: "latest"
        }
      }
    }
  }
}



It looks like the model selected the one available function and returned a function call for the `get_exchange_rate` function along with the parameters. And the parameters are in the correct format that we wanted. Hooray for getting structured responses from generative models!

In the next section, you'll use the information in the response to make an API request.


### 9. Make an API request

Recall that function calling in Gemini does not actually make the external API call for you. Rather, you are free to use any language, library, or framework that you'd like!

Here you'll use the `requests` library in Python to call the exchange rate REST API.

Let's unpack the response into a Python dictionary:


In [20]:
params = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    params[key[9:]] = value
params

{'from': 'AUD', 'date': 'latest', 'to': 'SEK'}

Now we can call requests or any other method:


In [21]:
import requests

url = f"https://api.frankfurter.app/{params['date']}"
api_response = requests.get(url, params=params)
api_response.text

'{"amount":1.0,"base":"AUD","date":"2025-01-22","rates":{"SEK":6.9066}}'

And we have our response from the REST API, with the latest exchange rate information from today. In the next section, we'll pass this information back to the model so that it can generate a relevant response for the user.


### 10. Generate a response

Finally, let's generate a response for the user by passing back the function response to the model in the next conversation turn:


In [23]:
response = model.generate_content(
    [
        Content(
            role="user",
            parts=[
                Part.from_text(
                    prompt
                    + """Give your answer in steps with lots of detail
            and context, including the exchange rate and date."""
                ),
            ],
        ),
        Content(
            role="function",
            parts=[
                Part.from_dict(
                    {
                        "function_call": {
                            "name": "get_exchange_rate",
                        }
                    }
                )
            ],
        ),
        Content(
            role="function",
            parts=[
                Part.from_function_response(
                    name="get_exchange_rate",
                    response={
                        "content": api_response.text,
                    },
                )
            ],
        ),
    ],
    tools=[exchange_rate_tool],
)

Once we pass the function response back to the model, it will respond to the user's prompt along with relevant information from the API response.


In [24]:
response.candidates[0].content.parts[0].text

'1. **Get the latest exchange rate:** I retrieved the latest exchange rate from our database.  The current exchange rate (as of January 22nd, 2025) is 1 Australian dollar (AUD) to 6.9066 Swedish krona (SEK). \n\n2. **Calculate the value of 500 AUD in SEK:**   Multiply the amount in AUD by the exchange rate:\n\n   500 AUD * 6.9066 SEK/AUD = 3453.30 SEK\n\n**Therefore, 500 Australian dollars are worth 3453.30 Swedish krona as of January 22nd, 2025.** \n'

### 11. View the full code example

At this point, you could put your Python code in a backend API using a Cloud Run service, a Cloud Function, or another Cloud service and deploy a frontend app that uses this backend API to perform model queries and API calls.

Here's the full code example for our final solution:


In [25]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

model = GenerativeModel("gemini-1.5-pro-001")

get_exchange_rate_func = FunctionDeclaration(
    name="get_exchange_rate",
    description="Get the exchange rate for currencies between countries",
    parameters={
        "type": "object",
        "properties": {
            "currency_date": {
                "type": "string",
                "description": "A date that must always be in YYYY-MM-DD format or the value 'latest' if a time period is not specified",
            },
            "currency_from": {
                "type": "string",
                "description": "The currency to convert from in ISO 4217 format",
            },
            "currency_to": {
                "type": "string",
                "description": "The currency to convert to in ISO 4217 format",
            },
        },
        "required": [
            "currency_from",
            "currency_date",
        ],
    },
)

exchange_rate_tool = Tool(
    function_declarations=[get_exchange_rate_func],
)

prompt = """What is the exchange rate from Australian dollars to Swedish krona?
How much is 500 Australian dollars worth in Swedish krona?"""

response = model.generate_content(
    prompt,
    tools=[exchange_rate_tool],
)

response.candidates[0].content

params = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    params[key[9:]] = value
params

import requests

url = f"https://api.frankfurter.app/{params['date']}"
api_response = requests.get(url, params=params)
api_response.text

response = model.generate_content(
    [
        Content(
            role="user",
            parts=[
                Part.from_text(
                    prompt
                    + """Give your answer in steps with lots of detail
            and context, including the exchange rate and date."""
                ),
            ],
        ),
        Content(
            role="function",
            parts=[
                Part.from_dict(
                    {
                        "function_call": {
                            "name": "get_exchange_rate",
                        }
                    }
                )
            ],
        ),
        Content(
            role="function",
            parts=[
                Part.from_function_response(
                    name="get_exchange_rate",
                    response={
                        "content": api_response.text,
                    },
                )
            ],
        ),
    ],
    tools=[exchange_rate_tool],
)


response.candidates[0].content.parts[0].text

'1. **Identify the exchange rate:** The provided exchange rate API response says that 1 Australian dollar (AUD) is equal to 6.9066 Swedish krona (SEK) on January 22, 2025. \n\n2. **Calculate the value of 500 AUD:** To find out how much 500 AUD is worth in SEK, we can multiply the amount in AUD by the exchange rate:\n\n   500 AUD * 6.9066 SEK/AUD = 3453.30 SEK\n\nTherefore, on January 22, 2025, 500 Australian dollars are worth **3453.30 Swedish krona**. \n'

In this implementation, we used two requests to the generative model: one request to generate a function call and another request to return the function response. Note that this is only one method of handling function calls and function responses with Gemini. You can also make additional function calls to get more information for your query, or use function calling with chat and asynchronous methods.

For additional code samples, refer to the [sample notebooks for function calling in Gemini](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/gemini/function-calling).
